In [51]:
import pandas as pd

In [52]:
# Function to calculate fiscal year in "YYYY-YYYY" format
def get_fiscal_year_range(date):
    start_month = 4
    if date.month >= start_month:
        return f"{date.year}-{date.year + 1}"
    else:
        return f"{date.year - 1}-{date.year}"

In [53]:
df_tax = pd.read_csv("../DATA/BRONZE/TaxReport/TaxReport_data.csv")

# Apply the function to add a new column for fiscal year
df_tax["fy"] = pd.to_datetime(df_tax["close_datetime"]).apply(get_fiscal_year_range)

df_tax = (
    df_tax.groupby(
        [
            "segment","stock_name",
        ]
    )
    .agg({"pnl_amount": "sum"})
    .sort_values(by=["segment","stock_name",])
    .reset_index()
)
print(df_tax["pnl_amount"].sum())
df_tax

10519.95


,segment,stock_name,pnl_amount
0,EQ,BHAGERIA,29.50
1,EQ,BPCL,-5510.45
2,EQ,GOLDBEES,0.42
3,EQ,HERANBA,3944.00
4,EQ,IDEA,114.00
5,EQ,INFY,182.45
6,EQ,LICI,-3294.75
7,EQ,NIFTYBEES,6.43
8,EQ,PNB,255.00
9,EQ,SBIN,994.00


In [54]:
# filepath of the CSV file
filepath = "../DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"

# Read the CSV file into a DataFrame
df_pnl = pd.read_csv(filepath)
df_pnl = (
    df_pnl.groupby(
        [
            "segment","stock_name",
        ]
    )
    .agg({"pnl_amount": "sum"})
    .sort_values(by=["segment","stock_name",])
    .reset_index()
)
print(df_pnl["pnl_amount"].sum())
df_pnl

16673.7


,segment,stock_name,pnl_amount
0,EQ,BHAGERIA,29.50
1,EQ,BPCL,-5510.45
2,EQ,GOLDBEES,0.42
3,EQ,HERANBA,3944.00
4,EQ,IDEA,114.00
5,EQ,INFY,182.45
6,EQ,LICI,-3294.75
7,EQ,NIFTYBEES,6.43
8,EQ,PNB,255.00
9,EQ,SBIN,994.00


In [55]:
# Merge DataFrames on 'stock_name'
df_merged = pd.merge(
    df_pnl, df_tax, on="stock_name", suffixes=("_pnl", "_tax"), how="outer"
)

# Compare pnl_amount from both DataFrames
df_merged["pnl_match"] = (
    df_merged["pnl_amount_pnl"] == df_merged["pnl_amount_tax"]
)
df_merged = df_merged[df_merged["pnl_match"]==False]
# Remove duplicate stock names, if necessary
# df_merged = df_merged.drop_duplicates(subset=['stock_name'])
df_merged.sort_values(by=["pnl_match", "stock_name"]).fillna(0).reset_index(
    drop=True
)

,segment_pnl,stock_name,pnl_amount_pnl,segment_tax,pnl_amount_tax,pnl_match
0,FO,NIFTY-CE-23500-27JUN2024,6375.00,FO,2600.00,False
1,FO,NIFTY-CE-24350-11JUL2024,-4103.75,FO,-5705.00,False
2,FO,NIFTY-CE-24400-11JUL2024,7187.50,FO,4260.00,False
3,FO,NIFTY-CE-24700-08AUG2024,-2750.00,0,0.00,False
4,FO,NIFTY-PE-24650-18JUL2024,-46.25,FO,-646.25,False
